<a href="https://colab.research.google.com/github/m-zayed5722/Miscellaneous-Projects/blob/main/LLM_Enricher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import time
from typing import Dict, List, Optional, Any, Union
from dataclasses import dataclass
import logging
from src.schema import MenuItem, MenuSchema
from pydantic import ValidationError


@dataclass
class LLMConfig:
    """Configuration for LLM integration"""
    model_name: str = "llama2"  # Default Ollama model
    base_url: str = "http://localhost:11434"
    timeout: int = 600  # 10 minutes timeout for large models
    max_retries: int = 3
    temperature: float = 0.1  # Low temperature for consistent structured output


class OllamaClient:
    """Client for interacting with Ollama API"""

    def __init__(self, config: LLMConfig = None):
        self.config = config or LLMConfig()
        self.logger = logging.getLogger(__name__)

    def is_available(self) -> bool:
        """Check if Ollama server is available"""
        try:
            response = requests.get(f"{self.config.base_url}/api/tags", timeout=5)
            return response.status_code == 200
        except Exception as e:
            self.logger.warning(f"Ollama not available: {e}")
            return False

    def list_models(self) -> List[str]:
        """List available models"""
        try:
            response = requests.get(f"{self.config.base_url}/api/tags", timeout=5)
            if response.status_code == 200:
                data = response.json()
                return [model['name'] for model in data.get('models', [])]
        except Exception as e:
            self.logger.error(f"Error listing models: {e}")
        return []

    def generate(self, prompt: str, model: str = None) -> Optional[str]:
        """Generate text using Ollama"""
        model = model or self.config.model_name

        payload = {
            "model": model,
            "prompt": prompt,
            "stream": False,
            "options": {
                "temperature": self.config.temperature
            }
        }

        for attempt in range(self.config.max_retries):
            try:
                self.logger.info(f"Sending request to {model} (timeout: {self.config.timeout}s)...")
                start_time = time.time()

                response = requests.post(
                    f"{self.config.base_url}/api/generate",
                    json=payload,
                    timeout=self.config.timeout
                )

                elapsed_time = time.time() - start_time
                self.logger.info(f"Response received in {elapsed_time:.2f} seconds")

                if response.status_code == 200:
                    result = response.json()
                    return result.get('response', '').strip()
                else:
                    self.logger.warning(f"Ollama API error: {response.status_code} - {response.text}")

            except requests.exceptions.Timeout as e:
                self.logger.warning(f"Attempt {attempt + 1} timed out after {self.config.timeout}s: {e}")
                if attempt < self.config.max_retries - 1:
                    self.logger.info("Retrying with exponential backoff...")
                    time.sleep(2 ** attempt)  # Exponential backoff
            except Exception as e:
                self.logger.warning(f"Attempt {attempt + 1} failed: {e}")
                if attempt < self.config.max_retries - 1:
                    time.sleep(2 ** attempt)  # Exponential backoff

        return None


class LLMEnricher:
    """LLM-based menu item enricher"""

    def __init__(self, config: LLMConfig = None):
        self.config = config or LLMConfig()
        self.client = OllamaClient(config)
        self.schema = MenuSchema()
        self.logger = logging.getLogger(__name__)

        # System prompt template
        self.system_prompt = """You are an expert food categorization assistant. Your task is to analyze menu item names and return structured, clean information.



In [ ]:
import sys
import os
sys.path.append('.')
from src.llm_enricher import get_enricher, LLMConfig
from src.baseline import BaselineClassifier
import json

def test_noodles_enrichment():
    """Test menu item enrichment specifically for noodles"""

    print("🍜 Testing Menu Item Enrichment for Noodles")
    print("=" * 50)

    # Configure to use real Ollama (not mock)
    config = LLMConfig(model_name="llama2", timeout=60)
    llm_enricher = get_enricher(config=config, use_mock=False)
    baseline = BaselineClassifier()

    # Test the specific example "noodles"
    test_item = "noodles"

    print(f"📝 Testing item: '{test_item}'")
    print("-" * 30)

    # Test LLM enrichment
    print("🤖 LLM (Ollama) Result:")
    try:
        llm_result = llm_enricher.enrich_item(test_item)
        if llm_result:
            print(f"   ✅ Name: {llm_result.item_name}")
            print(f"   ✅ Category: {llm_result.category}")
            print(f"   ✅ Cuisine: {llm_result.cuisine}")
            print(f"   ✅ Attributes: {llm_result.attributes if llm_result.attributes else 'None'}")
        else:
            print("   ❌ LLM enrichment failed")
    except Exception as e:
        print(f"   ❌ Error: {e}")

    print()

    # Test baseline for comparison
    print("📝 Baseline (Rule-based) Result:")
    try:
        baseline_result = baseline.classify_item(test_item)
        print(f"   ✅ Name: {baseline_result.item_name}")
        print(f"   ✅ Category: {baseline_result.category}")
        print(f"   ✅ Cuisine: {baseline_result.cuisine}")
        print(f"   ✅ Attributes: {baseline_result.attributes if baseline_result.attributes else 'None'}")
    except Exception as e:
        print(f"   ❌ Error: {e}")

    print("\n" + "=" * 50)
    print("✅ Test completed!")

if __name__ == "__main__":
    test_noodles_enrichment()

In [ ]:
from src.baseline_order_parser import BaselineOrderParser

def debug_double_bacon_matching():
    print("🔍 Debugging Double Bacon Quarter Pounder Matching")
    print("=" * 60)

    parser = BaselineOrderParser()

    # Find the Double Bacon Quarter Pounder item
    double_bacon_item = None
    for item in parser.menu_items:
        if "Double Bacon Quarter Pounder" in item.name:
            double_bacon_item = item
            break

    # Find ALL Quarter Pounder items and check them
    quarter_pounder_items = []
    for item in parser.menu_items:
        if "quarter pounder" in item.name.lower():
            quarter_pounder_items.append(item)

    test_text = "craving a mcchicken with large fries and medium sprite, mayo and ketchup included"
    print(f"Testing against: '{test_text}'")
    print(f"Found {len(quarter_pounder_items)} Quarter Pounder items:\n")

    for item in quarter_pounder_items:
        print(f"Item: {item.name}")
        print(f"Keywords: {item.keywords}")

        matched_keywords = []
        for keyword in item.keywords:
            if keyword in test_text.lower():
                matched_keywords.append(keyword)

        if matched_keywords:
            print(f"❌ Matched keywords: {matched_keywords}")
        else:
            print("✅ No keywords matched!")
        print("-" * 40)

if __name__ == "__main__":
    debug_double_bacon_matching()